In [ ]:
#| default_exp mingpt
%load_ext autoreload
%autoreload 2

In [ ]:
do_download:bool=False
if do_download:
    !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# | export
import torch
import pandas as pd

'Come you ladies and you gentlemen, a-listen to my song\n\nSing it to you right, but you might think it’s wrong\n\nJust a little glimpse of a story I’ll tell\n\n’Bout an East Coast city that you all know well\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nOld New York City is a friendly old town\n\nFrom Washington Heights to Harlem on down\n\nThere’s a-mighty many people all millin’ all around\n\nThey’ll kick you when you’re up and knock you when you’re down\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nIt’s a mighty long ways from the Golden Gate\n\nTo Rockefeller Plaza ’n’ the Empire State.\n\nMister Rockefeller sets up as high as a bird\n\nOld Mister Empire never says a word\n\nIt’s hard times from the country\n\nLivin’ down in New York town\n\n\n\nWell, it’s up in the mornin’ tryin’ to find a job of work\n\nStand in one place till your feet begin to hurt\n\nIf you got a lot o’ money you can make yourself merry\n\nIf you only got a nick

In [ ]:
is_dylan: bool = False
if is_dylan:
    df = pd.read_csv("./bob_dylan_lyrics.csv")
    text = "\n\n".join(df.lyrics.values.tolist())
else:
    with open("./input.txt", "r", encoding="utf-8") as f:
        text = f.read()

    len(text), text[:100]
text

'Come you ladies and you gentlemen, a-listen to my song\n\nSing it to you right, but you might think it’s wrong\n\nJust a little glimpse of a story I’ll tell\n\n’Bout an East Coast city that you all know well\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nOld New York City is a friendly old town\n\nFrom Washington Heights to Harlem on down\n\nThere’s a-mighty many people all millin’ all around\n\nThey’ll kick you when you’re up and knock you when you’re down\n\nIt’s hard times in the city\n\nLivin’ down in New York town\n\n\n\nIt’s a mighty long ways from the Golden Gate\n\nTo Rockefeller Plaza ’n’ the Empire State.\n\nMister Rockefeller sets up as high as a bird\n\nOld Mister Empire never says a word\n\nIt’s hard times from the country\n\nLivin’ down in New York town\n\n\n\nWell, it’s up in the mornin’ tryin’ to find a job of work\n\nStand in one place till your feet begin to hurt\n\nIf you got a lot o’ money you can make yourself merry\n\nIf you only got a nick

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
stoi = {s: i for i, s in enumerate(chars)}
itos = {i: s for i, s in enumerate(chars)}


def encode(s: str) -> list[int]:
    return [stoi[c] for c in s]


def decode(l: list[int]) -> str:
    return "".join([itos[i] for i in l])


decode(encode("hello world"))

'hello world'

In [ ]:
# torch.tensor()
data = torch.tensor(encode(text), dtype=torch.long)
data[:100]
n = int(0.9 * len(text))
train = data[:n]
val = data[n:]

In [ ]:
block_size = 8

In [ ]:
x = train[:block_size]
y = train[1 : block_size + 1]
x, y
for t in range(block_size):
    ctx = x[: t + 1]
    tgt = y[t]
    # print(t, ctx, tgt)

In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split: str):
    data = train if split == "train" else val
    ix = torch.randint(len(data) - block_size, (batch_size,))
    print(ix)
    # ix are random starting points of each batch
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

print(x)
for b in range(batch_size):
    for t in range(block_size):
        ctx = xb[b, 0 : t + 1]
        tgt = yb[b, t]
        # print(ctx,"->", tgt)

tensor([465733,  94837, 447242, 141694])
tensor([26, 65, 63, 55,  2, 75, 65, 71])


In [ ]:
emb = torch.nn.Embedding(vocab_size, vocab_size)
emb(torch.tensor([4, 0, 2]))

In [ ]:
emb.weight[4]

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export("mingpt.ipynb")